# Slicing by data from an external dictionary (continued)

In [ ]:
import pandas as pd

You remembered how to access an external variable in `query()` and check if the value of column a is in the list.
These tricks are combined when a list is stored in an external variable. To find out if the value of column a occurs in the list `our_list = [1, 2, 3]`, write the query: `'a in @our_list'`.

In [ ]:
df = pd.DataFrame({
    'a': [0, 1, 10, 11, 12], 
    'b': [5, 4, 3, 2, 1],
    'c': ['X', 'Y', 'Y', 'Y', 'Z']
})

In [ ]:
df

In [ ]:
our_list = [1, 2]

df.query('a in @our_list')

So far everything is working normally. However, not only numbers, strings or lists are stored in a variable, but also more complex structures: `dictionaries`, `Series`, `dataframes`. See how the condition works for building a slice with them.
What if the external variable does not store a `list`, but a `dictionary`? Recall that a dictionary is a data structure consisting of a key and a value. Let in the variable our_dict key-value pairs: `our_dict = {0: 10, 1: 11, 2: 12}`. Then the query `a in @our_dict` will check whether the values of column a are in the dictionary keys: `0`, `1`, `2`.

In [ ]:
our_dict = {0: 10, 1: 11, 2: 12} 

df.query('a in @our_dict')

When a `Series` object is stored in a variable, the a in `@our_series` construct will check if the `list` contains values, not `indixes`:

In [ ]:
our_series=pd.Series([10,11,12])

df.query('a in @our_series')

If it is necessary to check the occurrence in the `index`, this is indicated explicitly by appending the `index` with a dot: `'a in @our_series.index'`.

In [ ]:
our_series = pd.Series([10,11,12])

df.query('a in @our_series.index')

When dealing with a `DataFrame` object, the occurrence of an index is checked in the same way as in a Series - by prefixing index with a dot to the name of the dataframe:

In [ ]:
df = pd.DataFrame({
    'a': [0, 1, 10, 11, 12],
    'b': [5, 4, 3, 2, 1],
    'c': ['X', 'Y', 'Y', 'Y', 'Z']
})

our_df = pd.DataFrame ({
    'a1': [2, 4, 6],
    'b1': [3, 2, 2],
    'c1': ['A', 'B', 'C']
})

df.query('a in @our_df.index')

To check if a column is included, its name is passed. Let's build a slice from the values of column `b` of the first dataframe, equal to the elements of column `b1` of the second dataframe:

In [ ]:
df.query('b in @our_df.b1')

# Adding a column

See how to add a column from one dataframe to another:

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3, 3, 3], 
                    'b': ['Q', 'R', 'S', 'T', 'U']})
df2 = pd.DataFrame({'c': [3, 4, 5, 6, 7], 
                    'd': ['V', 'W', 'X', 'Y', 'Z'], 
                    'e': [3, 3, 3, 3, 3]})

display(df1)
display(df2) 

To add a column from `df2` to `df1`, create a new column in `df1` and assign it the values of column `df2`:

In [ ]:
df1['new'] = df2['d']
display(df1) 

If the new column were already in `df1`, then all its elements would be <font color="#f00505">deleted</font>, and new ones would be written instead:

Seems simple: pandas copies the column from `df2` and pastes it into `df1`.
However, everything is more complicated. For each row in the first dataframe, pandas looks for a "pair" - the row with the same `index` in the second dataframe. Finds and takes a value from this string. In our case, the indexes in `df1` and `df2` matched, and everything seemed to be a simple copying of rows in order.

Let's replace the `indexes` in the second dataframe with the values of column `c`. Then we assign the new column in the first dataframe to the values of the `d` column in `df2`:

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3, 3, 3], 
                    'b': ['Q', 'R', 'S', 'T', 'U']})
df2 = pd.DataFrame({'c': [3, 4, 5, 6, 7], 
                    'd': ['V', 'W', 'X', 'Y', 'Z'], 
                    'e': [3, 3, 3, 3, 3]})
df2.set_index('c', inplace=True)

In [ ]:
df1['new'] = df2['d']

display(df1) 

The indices in `df1` and `df2` are no longer the same. Assignment occurs only by matching indexes. There are no indexes `0`, `1`, `2` in `df2` - these lines of the final dataframe turned out to be `NaN`. And the rows with indices `3` and `4` contain the values that were in `df2['d']` in the rows with indices `3` and `4`.
If there are duplicate indexes in `df1`, then the value from `df2['d']` will be copied multiple times.

The number of rows in `df2` need not be the same as the number of rows in `df1`. If there are not enough values in `df2`, it will be `None`. And there will be superfluous ones - they simply will not fall into the updated dataframe.
But duplicate values in the `df2` index will lead to an error. In this case, pandas will not understand which value should be substituted into `df1`.

A separate column can be created without a dataframe, in `Series` it will be a set of values with indexes. When trying to assign an object with indices, pandas will pick up the rows corresponding to the indices:

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3, 3, 3], 
                    'b': ['Q', 'R', 'S', 'T', 'U']})
df1.set_index('a', inplace=True)

series = pd.Series([1, 2, 3, 4, 5])

In [ ]:
df1['new'] = series

display(df1) 

If you pass to the column a list of values stored not as a `Series`, but, for example, as a `list`, the assignment will go in row order:

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3, 3, 3], 
                    'b': ['Q', 'R', 'S', 'T', 'U']})
df1.set_index('a', inplace=True)

list_values = [1, 2, 3, 4, 5]

In [ ]:
df1['new'] = list_values

display(df1) 

# Merging data from two tables

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3, 5], 'b': ['A', 'B', 'C', 'D']})
df2 = pd.DataFrame({'a': [2, 2, 2, 3], 'c': ['E', 'F', 'G', 'H']})

joining tables in python follows the same rules as in SQL, just like in SQL there are two methods for joining tables `merge` and `join`

Using the `merge()` method, we merge dataframe rows by matching values of column `a`:

In [ ]:
df1.merge(df2, on='a')

The final table was formed from matches in column a of records. This type of union is called `inner`. It collects data from the inner area (which is in both dataframes). In `merge()` , the inner type works by default.

Its opposite is the `outer` merge type. It combines data from the outer common area - such that is in at least one of the dataframes. The join mode is specified by the how parameter.

In [ ]:
df1.merge(df2, on='a', how='outer') 

#### this image should help you in the future

![jupyter](./pict/join.png)

The list is ready, we haven't missed a single entry. Please note that if there is no data, the cells are NaN.
The merge mode `'left'` specifies that the result of the merge must include all rows from the left dataframe:

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3, 5], 'b': ['A', 'B', 'C', 'D']})
df2 = pd.DataFrame({'a': [2, 2, 2, 3], 'b': ['E', 'F', 'G', 'H']})

In [ ]:
df1.merge(df2, on='a', how='left') 

In a similar `'right'` mode, all matching rows and the right dataframe will be preserved.
Please note that in the result table of the `merge()` method, `_x` and `_y` have been added to the column names. The endings of the column names are set in the suffixes parameter:

In [ ]:
df1.merge(df2, on='a', how='left', suffixes=('_df1', '_df2')) 

If the index column is named, its name can also be passed to the on parameter. You can merge on several columns at once - just pass the list to the `on` parameter.

The `join()` method is similar to `merge()`. Without the on parameter, this `join()` will look for index matches in the first and second dataframes. If you pass a column to the on parameter, then `join()` will find it in the first dataframe and start comparing it with the index of the second. Unlike `merge()`, the default `join()` merge type is `how='left'`. And the suffixes parameter is divided into two independent ones: `lsuffix` and `rsuffix`. You can also join more than two tables using the `join()` method: their set is passed as a list instead of the second dataframe.

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3, 4], 'b': ['A', 'B', 'C', 'D']})
df2 = pd.DataFrame({'a': [2, 2, 2, 2], 'c': ['E', 'F', 'G', 'H']})

df1.join(df2, on='a', rsuffix='_y')